In [7]:
import serial
import threading
import time
import csv

# --- KONFIGURASI PORT DAN DURASI ---
ESP32_PORT = 'COM9'   # Port yang akan dicari oleh skrip
BAUD_RATE = 115200
IDLE_DURATION_S = 5
ACTIVE_DURATION_S = 60
TOTAL_LOG_DURATION_S = IDLE_DURATION_S + ACTIVE_DURATION_S
# ------------------------------------

# --- KONFIGURASI EKSPERIMEN ---
# UBAH VARIABEL DI BAWAH INI SESUAI DENGAN PENGUJIAN ANDA
# =======================================================
KASUS = 'E'
MODEL_NO = '4'
TIPE = 'Float'
MCU = 'ESP32'
# =======================================================
# ------------------------------------

# Event untuk memberi sinyal berhenti ke thread
stop_event = threading.Event()

# (BARU) Event untuk memberi sinyal bahwa koneksi sudah berhasil
connection_established_event = threading.Event()

def log_power_data(port, baud, filename, connection_event):
    """Fungsi ini akan terus mencari port, memberi sinyal, lalu mencatat data."""
    ser = None
    print(f"--- Logger Siap ---")
    
    while not stop_event.is_set():
        try:
            ser = serial.Serial(port, baud, timeout=1)
            print(f"\n✅ Port {port} terdeteksi! Terhubung ke ESP32.")
            
            # (BARU) Kirim sinyal ke thread utama bahwa koneksi berhasil
            connection_event.set()
            
            break 
        except serial.SerialException:
            print(f"\r⏳ Mencari port {port}... Harap hubungkan perangkat Anda.", end="")
            time.sleep(1)
            continue
            
    if not ser:
        print("\nProses dihentikan sebelum port ditemukan.")
        return

    print(f"Mulai mencatat data daya ke file: {filename}")
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Bus Voltage (V)', 'Current (mA)', 'Power (mW)'])
        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data = line.split(',')
                    if len(data) == 3:
                        writer.writerow([float(data[0]), float(data[1]), float(data[2])])
            except (UnicodeDecodeError, ValueError, serial.SerialException):
                break
    
    ser.close()
    if not stop_event.is_set():
        print(f"\n⚠️ Koneksi terputus. Data yang tersimpan mungkin tidak lengkap.")
    else:
        print(f"\n✅ Sesi logging daya selesai. Data disimpan.")

if __name__ == "__main__":
    power_log_file = f"{KASUS}_{MODEL_NO}_{TIPE}_{MCU}_power.csv"
    
    # Masukkan 'connection_established_event' sebagai argumen baru untuk thread
    power_thread = threading.Thread(target=log_power_data, 
                                    args=(ESP32_PORT, BAUD_RATE, power_log_file, connection_established_event))
    power_thread.start()

    # (BARU) Thread utama akan berhenti di sini dan menunggu sinyal
    print("Menunggu koneksi perangkat...")
    connection_established_event.wait() 
    
    # Setelah sinyal diterima, baru mulai proses hitungan mundur
    print(f"\n🚀 Logging dimulai. Fase Idle: {IDLE_DURATION_S}d, Fase Aktif: {ACTIVE_DURATION_S}d.")
    print(f"Total durasi logging: {TOTAL_LOG_DURATION_S} detik. Harap tunggu...")
    
    time.sleep(TOTAL_LOG_DURATION_S)

    print("\n⏳ Waktu habis. Menghentikan sesi logging daya...")
    stop_event.set()

    power_thread.join()

    print("\n🎉 Proses selesai!")

--- Logger Siap ---
Menunggu koneksi perangkat...

✅ Port COM9 terdeteksi! Terhubung ke ESP32.
Mulai mencatat data daya ke file: E_4_Float_ESP32_power.csv

🚀 Logging dimulai. Fase Idle: 5d, Fase Aktif: 60d.
Total durasi logging: 65 detik. Harap tunggu...

⏳ Waktu habis. Menghentikan sesi logging daya...

✅ Sesi logging daya selesai. Data disimpan.

🎉 Proses selesai!


In [8]:
import pandas as pd
import os
import glob

# Ganti dengan path folder kamu
folder_path = "."  # contoh: "/mnt/data" atau "C:/Users/NamaKamu/Documents"

# Cari semua file CSV di folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Dictionary untuk menyimpan hasil
results = {}

for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)

        # Cari kolom yang mengandung kata 'power' atau 'watt'
        power_cols = [col for col in df.columns if 'power' in col.lower() or 'watt' in col.lower()]

        if power_cols:
            avg_power = df[power_cols[0]].mean()
            results[os.path.basename(file_path)] = avg_power
        else:
            results[os.path.basename(file_path)] = "Kolom daya tidak ditemukan"

    except Exception as e:
        results[os.path.basename(file_path)] = f"Error membaca file: {e}"

# Tampilkan hasil
print("Rata-rata konsumsi daya masing-masing file:\n")
for file, avg in results.items():
    print(f"{file}: {avg}")



Rata-rata konsumsi daya masing-masing file:

A_1_Float_RP2350_power.csv: 96.34571890145396
A_1_F_ESP32_power.csv: 228.36220472440945
A_1_Int_RP2350_power.csv: 96.38095238095238
A_1_I_ESP32_power.csv: 233.7987421383648
A_2_Float_RP2350_power.csv: 97.32794830371567
A_2_F_ESP32_power.csv: 229.72012578616352
A_2_Int_RP2350_power.csv: 96.47021943573668
A_2_I_ESP32_power.csv: 236.56062992125985
A_3_Float_RP2350_power.csv: 98.49270664505673
A_3_F_ESP32_power.csv: 239.51572327044025
A_3_Int_RP2350_power.csv: 96.52664576802508
A_3_I_ESP32_power.csv: 236.27044025157232
A_4_Float_RP2350_power.csv: 100.85945072697899
A_4_F_ESP32_power.csv: 281.17295597484275
A_4_Int_RP2350_power.csv: 98.1128526645768
A_4_I_ESP32_power.csv: 284.28930817610063
B_1_Float_RP2350_power.csv: 96.35736677115987
B_1_F_ESP32_power.csv: 231.9748427672956
B_1_Int_RP2350_power.csv: 95.91236306729265
B_1_I_ESP32_power.csv: 231.49056603773585
B_2_Float_RP2350_power.csv: 95.7373417721519
B_2_F_ESP32_power.csv: 232.5984251968504
B